# Imports

In [1]:
import pandas as pd
import numpy as np
from datasets import load_dataset, Dataset, DatasetDict, load_from_disk
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    TrainingArguments, Trainer,
    DataCollatorForLanguageModeling,
    DataCollatorWithPadding
)
from peft import PeftModel, get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training
from transformers import BitsAndBytesConfig
import torch
import wandb

from utils import tokenize_dataset_for_domain_bound_qna
# from prompt_templates import domain_bound_qna_prompt_template as prompt_template

/home/super_admin/hasindu/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Configs

In [2]:
data_path = "../data/domain_bound_data/v4/"
train_data_path = data_path + "train.csv"
val_data_path = data_path + "val.csv"

max_len = 512

base_model_path = "../models/phi_qna_finetuned_attempt_3/final_pretrained_2"

model_id = "microsoft/Phi-3.5-mini-instruct"

model_output_dir = "../models/phi_domain_bound_qna_finetuned_attempt_6"

# Hyperparameters

In [3]:
lora_r = 64
lora_alpha = 128
lora_target_modules = ["qkv_proj"]
batch_size = 32
quantization = None
lora_dropout = 0.05
epochs = 3
learning_rate = 8e-6
warmup_steps = 300
learning_rate_scheduler = "linear"

# Dataset

In [4]:
prompt_template = """
# Instruction:
Assume you are an excellent doctor. Using your knowledge, answer the question given below.

# Question: {question}

# Answer: """
prompt_template = prompt_template.strip()
print(prompt_template)

# Instruction:
Assume you are an excellent doctor. Using your knowledge, answer the question given below.

# Question: {question}

# Answer:


In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

In [6]:
val_df = pd.read_csv(val_data_path)
train_df = pd.read_csv(train_data_path)

val_set = tokenize_dataset_for_domain_bound_qna(tokenizer, val_df[["question", "answer"]], prompt_template, max_len)
train_set = tokenize_dataset_for_domain_bound_qna(tokenizer, train_df[["question", "answer"]], prompt_template, max_len)

Map: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2300/2300 [00:01<00:00, 1538.83 examples/s]


In [ ]:
train_set[0]

In [ ]:
train_set.save_to_disk(data_path + "tokenized")
# train_set = load_from_disk(data_path + "tokenized")

In [7]:
wandb.init(
    project="med-domain_bound_qna_finetune",
    name="attempt_5",
    config={
        "model": model_id,
        "lora_r": lora_r,
        "lora_alpha": lora_alpha,
        "batch_size": batch_size,
        "epochs": epochs,
        "quantization": quantization,
        "lora_target_modules": lora_target_modules,
        "learning_rate_scheduler": learning_rate_scheduler,
        "warmup_steps": warmup_steps
    }
)

wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: hasindumadushan325 (hasindumadushan325-university-of-peradeniya) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=quantization=="4bit",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
# temp = AutoModelForCausalLM.from_pretrained(
#     model_id,    # quantization_config=bnb_config,
#     device_map="auto",
#     trust_remote_code=True
# )


In [8]:
model = AutoModelForCausalLM.from_pretrained(
    base_model_path,    # quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=False
)

Loading checkpoint shards: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.95it/s]


In [9]:
model.gradient_checkpointing_enable()
# model = prepare_model_for_kbit_training(model)

In [10]:
lora_config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    target_modules=lora_target_modules,
    lora_dropout=lora_dropout,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)

# Train

In [11]:
training_args = TrainingArguments(
    output_dir=model_output_dir,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epochs,
    eval_strategy="epoch",  # ✅ eval at each epoch
    save_strategy="epoch",
    logging_steps=50,
    learning_rate=learning_rate,
    lr_scheduler_type=learning_rate_scheduler,
    warmup_steps=warmup_steps,
    fp16=True,
    report_to="wandb",
    run_name="attempt_6",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss"
)

In [12]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=val_set,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer, padding=False)
)

/tmp/ipykernel_85916/1780078394.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [13]:
trainer.train()

trainer.save_model(model_output_dir + "/final")
tokenizer.save_pretrained(model_output_dir + "/final")

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Epoch,Training Loss,Validation Loss
1,3.543100,1.691156
2,3.170900,1.641109
3,1.865700,1.679964


('../models/phi_domain_bound_qna_finetuned_attempt_6/final/tokenizer_config.json',
 '../models/phi_domain_bound_qna_finetuned_attempt_6/final/special_tokens_map.json',
 '../models/phi_domain_bound_qna_finetuned_attempt_6/final/tokenizer.json')

## Merge model with lora weights

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_path,    # quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

finetuned_model = PeftModel.from_pretrained(base_model, model_output_dir + "/final")
merged_model = finetuned_model.merge_and_unload()

In [ ]:
merged_model.save_pretrained(model_output_dir +  "/final_pretrained", safe_serialization=True)